In [1]:
mkdir model

In [3]:
import os
import tarfile

# Specify the location to save the model artifact locally
local_model_dir = "extracted_model"

# Create the directory if it doesn't exist
os.makedirs(local_model_dir, exist_ok=True)

# Specify the S3 model artifact location
s3_model_location = "s3://aicsproject/model_epoch_15_sayesh.tar.gz"

# Download the model artifact from S3
!aws s3 cp {s3_model_location} {local_model_dir}

# Unzip the model artifact
with tarfile.open(os.path.join(local_model_dir, "model_epoch_15_sayesh.tar.gz"), "r:gz") as tar:
    tar.extractall(path=local_model_dir)

# Now you have extracted the model artifact, you can proceed with deploying it


download: s3://aicsproject/model_epoch_15_sayesh.tar.gz to extracted_model/model_epoch_15_sayesh.tar.gz


In [10]:
# import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torchvision.models import resnet18
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os

In [11]:

# Define your PyTorch model
class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.resnet = resnet18(pretrained=True)
        self.fc = nn.Linear(1000, 2)  # Assuming binary classification

    def forward(self, x):
        x = self.resnet(x)
        x = self.fc(x)
        return x

In [12]:
# Define a dataset class
class CustomDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.images = os.listdir(data_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = os.path.join(self.data_dir, self.images[idx])
        image = Image.open(img_name)
        if self.transform:
            image = self.transform(image)
        return image

In [ ]:
import tarfile

# Extract the contents of the .tar.gz file
with tarfile.open('/model/model_epoch_15_sayesh.tar.gz', 'r:gz') as tar:
    tar.extractall('/home/ec2-user/Sagemaker/model/')  # Extract to the model directory

# Load the model from the extracted file
model = CustomModel()
model_path = '/home/ec2-user/Sagemaker/model/model_epoch_15_sayesh.pth'  # Assuming the model file is model_epoch_15_sayesh.pth
model.load_state_dict(torch.load(model_path))
model.eval()


In [ ]:
# Define a function to handle input and output serialization/deserialization
def input_fn(serialized_input_data, content_type):
    if content_type == 'image/jpeg':
        input_data = Image.open(io.BytesIO(serialized_input_data))
        input_data = transform(input_data)
        return input_data
    else:
        raise ValueError(f'Unsupported content type: {content_type}')

def output_fn(prediction_output, accept):
    return str(prediction_output)

In [ ]:
# Deploy the model to SageMaker
from sagemaker.pytorch import PyTorchModel

pytorch_model = PyTorchModel(model_data='s3://aicsproject/model_epoch_15_sayesh.tar.gz',
                              role=role,
                              framework_version='1.7.1',
                              entry_point='inference.py',
                              source_dir='source_dir')  # Directory containing your inference script
predictor = pytorch_model.deploy(instance_type='ml.t2.medium', initial_instance_count=1)